In [ ]:
import pandas as pd
import numpy as np
import category_encoders as ce
import shap
import eli5
from eli5.sklearn import PermutationImportance
from scipy.stats import randint, uniform
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/justin-hsieh/powerlifting-project/master/file6.csv')
df = df.drop(columns='Unnamed: 0')

df.head()


In [3]:
X_train, X_test  = train_test_split(df, test_size=0.20,  
                              random_state=42)
X_train.shape, X_test.shape

((320788, 5), (80198, 5))

In [4]:
def wrangle(df):
    df = df.copy()
    df['Sex'] = df['Sex'].astype(str)
    df['Sex'] = df['Sex'].map({'M':0,'F':1})
    
    df['Equipment'] = df['Equipment'].astype(str)
    df['Equipment'] = df['Equipment'].map({
                                                'Raw':0,'Single-ply':1,
                                                'Wraps':2,'Multi-ply':3
                                              })
    
    print(df.shape)
    return df
    

In [5]:
Xtrain = wrangle(X_train)
Xtest = wrangle(X_test)


target = 'TotalKg'

X_train = Xtrain.drop(columns=target)
y_train = Xtrain[target]
X_test = Xtest.drop(columns=target)
y_test = Xtest[target]

(320788, 5)
(80198, 5)


In [12]:
pipeline = make_pipeline(
     RandomForestRegressor(max_depth=10,
                           random_state=42,
                           max_features=0.07968793248374506,
                           n_estimators=103,
                           n_jobs=-1
                        )
)



In [13]:
model = pipeline.fit(X_train,y_train)

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print('Test MAE', mae)

Test MAE 67.6608299321711


In [14]:
def predict(Sex, Age, Bodyweight, Equipment):
    df = pd.Dataframe(
        data=[[Sex,Equipment,Age,BodyweightKg]],
        
        columns=['Sex','Equipment','Age','Bodyweight']
                    )
    
    pred = model.predict(df)[0]
    
    result = f'Based on the inputs, your predicted place is: {pred:,.0f}\n\n'
        
    return result
    
from joblib import dump
dump(model, 'model.joblib')

['model.joblib']